#### Подготовка Данных

##### EDA

In [ ]:
from pathlib import Path
import pandas as pd

file_path = Path.cwd() / "ml" / "data" / "raw" / "IBM HR Data new.csv"
print("cwd:", Path.cwd())
print("try:", file_path)

if not file_path.exists():
    file_path = Path.cwd().parent / "ml" / "data" / "raw" / "IBM HR Data new.csv"
    print("try2:", file_path)

print("exists:", file_path.exists())

df = pd.read_csv(file_path)
df.shape
df.info()

miss = (df.isna().mean() * 100).sort_values(ascending=False)
miss[miss > 0].head(15)

df["Attrition"].value_counts(dropna=False).head(20)


##### Data preparation

In [ ]:

obj_cols = df.select_dtypes(include="object").columns
num_like = []
for c in obj_cols:
    s = df[c].dropna().astype(str).str.replace(",", ".", regex=False).str.strip()
    share = pd.to_numeric(s, errors="coerce").notna().mean()
    if share > 0.9:
        num_like.append(c)

print("object -> numeric candidates:", num_like)

for c in num_like:
    s = df[c].astype(str).str.replace(",", ".", regex=False).str.strip()
    df[c] = pd.to_numeric(s, errors="coerce")

num_cols = df.select_dtypes(include=["number"]).columns
cat_cols = df.select_dtypes(include="object").columns

df[num_cols] = df[num_cols].fillna(df[num_cols].median(numeric_only=True))

for c in cat_cols:
    if df[c].isna().any():
        df[c] = df[c].fillna(df[c].mode(dropna=True)[0])

miss_after = (df.isna().mean() * 100).sort_values(ascending=False)
print("top missing after:", miss_after.head(10).to_dict())

df = df.drop(columns=["EmployeeCount", "EmployeeNumber", "Application ID"])
df["target"] = (df["Attrition"] == "Voluntary Resignation").astype(int)


##### Export: сохранить processed

In [9]:

out = Path("../ml/data/processed/hr_processed.csv")
out.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out, index=False)

print("saved:", out.resolve())
print("shape:", df.shape)
print("target share:", df["target"].value_counts(normalize=True).to_dict())

cwd: /home/pydev/Desktop/courses/AI-project/notebooks
try: /home/pydev/Desktop/courses/AI-project/notebooks/ml/data/raw/IBM HR Data new.csv
try2: /home/pydev/Desktop/courses/AI-project/ml/data/raw/IBM HR Data new.csv
exists: True


/tmp/ipykernel_14765/780465669.py:14: DtypeWarning: Columns (5,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


object -> numeric candidates: ['DistanceFromHome', 'EmployeeCount', 'EmployeeNumber', 'Application ID', 'HourlyRate', 'JobSatisfaction', 'MonthlyIncome', 'PercentSalaryHike']
top missing after: {'Age': 0.0, 'Attrition': 0.0, 'BusinessTravel': 0.0, 'DailyRate': 0.0, 'Department': 0.0, 'DistanceFromHome': 0.0, 'Education': 0.0, 'EducationField': 0.0, 'EmployeeCount': 0.0, 'EmployeeNumber': 0.0}
saved: /home/pydev/Desktop/courses/AI-project/ml/data/processed/hr_processed.csv
shape: (23436, 35)
target share: {0: 0.8417392046424305, 1: 0.15826079535756954}
